In [1]:
from keras.applications.vgg16 import VGG16
from keras.datasets import cifar10
from keras.utils import to_categorical
import matplotlib.pyplot  as plt

In [35]:
# Load and preprocess the CIFAR-10 dataset
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255.0
x_test /= 255.0

In [36]:
y_test.shape

(10000, 1)

In [37]:
x_train.ndim

4

In [38]:
x_train.shape

(50000, 32, 32, 3)

In [8]:
# Load the pre-trained VGG16 model without the top layer
model = VGG16(weights='imagenet', include_top=False, input_shape=(32, 32, 3))

58889256/58889256 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [40]:
# Perform one-hot encoding on the labels
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)

In [42]:
y_test.shape

(10000, 10)

In [10]:
# Extract features using the pre-trained VGG16 model
train_features = model.predict(x_train)
test_features = model.predict(x_test)

1563/1563 ━━━━━━━━━━━━━━━━━━━━ 629s 402ms/step
313/313 ━━━━━━━━━━━━━━━━━━━━ 125s 399ms/step


In [12]:
train_features.shape

(50000, 1, 1, 512)

In [11]:
test_features.shape

(10000, 1, 1, 512)

In [33]:
# Flatten the extracted features
train_features = train_features.reshape(train_features.shape[0], -1)
test_features = test_features.reshape(test_features.shape[0], -1)

(49999, 1000)

In [34]:
y_test.shape

(9999, 1000)

In [32]:
y_train.shape

(49999, 1000)

In [14]:
test_features.shape

(10000, 512)

In [15]:
train_features.shape [1]

512

In [16]:
test_features.shape[1]

512

In [19]:
# Build a classifier on top of the extracted features
from keras.models import Sequential
from keras.layers import Dense, Dropout

In [25]:
classifier = Sequential()
classifier.add(Dense(256, activation='relu', input_dim=train_features.shape[1]))
classifier.add(Dense(128, activation='relu'))
classifier.add(Dense(64, activation='relu'))
classifier.add(Dense(10, activation='softmax'))

In [43]:
# Compile and train the classifier
classifier.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
classifier.fit(train_features, y_train, batch_size=32, epochs=10, validation_data=(test_features, y_test))

Epoch 1/10
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 10s 5ms/step - accuracy: 0.4602 - loss: 1.5246 - val_accuracy: 0.5791 - val_loss: 1.2066
Epoch 2/10
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 9s 5ms/step - accuracy: 0.5935 - loss: 1.1661 - val_accuracy: 0.5815 - val_loss: 1.1897
Epoch 3/10
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 11s 5ms/step - accuracy: 0.6150 - loss: 1.1018 - val_accuracy: 0.5892 - val_loss: 1.1745
Epoch 4/10
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 9s 6ms/step - accuracy: 0.6405 - loss: 1.0422 - val_accuracy: 0.6058 - val_loss: 1.1194
Epoch 5/10
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 7s 4ms/step - accuracy: 0.6543 - loss: 0.9853 - val_accuracy: 0.6105 - val_loss: 1.1212
Epoch 6/10
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 9s 5ms/step - accuracy: 0.6653 - loss: 0.9510 - val_accuracy: 0.6128 - val_loss: 1.1248
Epoch 7/10
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 11s 6ms/step - accuracy: 0.6845 - loss: 0.8978 - val_accuracy: 0.6133 - val_loss: 1.1439
Epoch 8/10
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 9s 5ms/step - accuracy: 0.6989 - loss: 0.8613

In [44]:
classifier.evaluate(test_features, y_test)

313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.6155 - loss: 1.1549


[1.1614667177200317, 0.6161999702453613]